# Blockchain for School Marks

This notebook demonstrates the creation of a simple blockchain to store school marks. Each student's data is represented as a block in the blockchain.

In [ ]:
# Imports
import os
import pandas as pd
import hashlib as hasher
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

# Constants
CSV_FILE = "Marks.csv"

# Load and clean marks data
def load_and_clean_data(file_name):
    if not os.path.exists(file_name):
        raise FileNotFoundError(f"{file_name} not found in directory.")
    
    df = pd.read_csv(file_name)
    
    subject_map = {
        "math": "Mathematics", "Math": "Mathematics", "maths": "Mathematics",
        "sci": "Science", "Sci": "Science",
        "eng": "English", "Eng": "English"
    }
    df["Subject"] = df["Subject"].str.strip().replace(subject_map)
    df.dropna(subset=["Marks"], inplace=True)

    required = {"Student", "Subject", "Marks", "Exam"}
    if not required.issubset(df.columns):
        raise ValueError(f"CSV must contain columns: {required}")
    
    return df

# Blockchain Block
class Block:
    def __init__(self, index, timestamp, data, previous_hash):
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash
        self.hash = self.hash_block()
    
    def hash_block(self):
        sha = hasher.sha256()
        sha.update((str(self.index) + str(self.timestamp) +
                    str(self.data) + str(self.previous_hash)).encode('utf-8'))
        return sha.hexdigest()

# Blockchain Helpers
def create_genesis_block():
    return Block(0, dt.datetime.now(), "Genesis Block", "0")

def next_block(last_block, data):
    return Block(last_block.index + 1, dt.datetime.now(), data, last_block.hash)

# Build Blockchain from DataFrame
def build_blockchain(df):
    chain = [create_genesis_block()]
    last = chain[0]

    for _, row in df.iterrows():
        data = f"Student Record: {row.to_dict()}"
        block = next_block(last, data)
        chain.append(block)
        last = block
        print(f"Block #{block.index} added with hash: {block.hash}")
    
    return chain

# Convert blockchain to DataFrame
def blockchain_to_df(chain):
    return pd.DataFrame([{
        "Index": b.index,
        "Timestamp": b.timestamp,
        "Data": b.data,
        "Previous Hash": b.previous_hash,
        "Hash": b.hash
    } for b in chain])

# Search student records
def search_student_by_name(name, blockchain):
    print(f"\nSearching for: {name}\n" + "-"*40)
    found = False
    for block in blockchain[1:]:
        if name.lower() in block.data.lower():
            found = True
            print(f"Block #{block.index} | Data: {block.data} | Time: {block.timestamp}")
    if not found:
        print("No records found.")

# Performance comparison
def analyze_performance(df):
    avg = df.groupby("Student")["Marks"].mean().reset_index().rename(columns={"Marks": "Average_Marks"})
    df = df.merge(avg, on="Student")
    df["Performance_Trend"] = df.apply(
        lambda row: "Above Average" if row["Marks"] > row["Average_Marks"]
        else "Below Average" if row["Marks"] < row["Average_Marks"]
        else "At Average", axis=1
    )
    return df

# Visualization
def plot_performance(df):
    sns.set(style="whitegrid")
    plt.figure(figsize=(12, 6))
    sns.barplot(data=df, x="Student", y="Marks", hue="Subject", ci=None)
    plt.title("Student Performance by Subject")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

def plot_subject_pie(df):
    total = df.groupby("Subject")["Marks"].sum()
    plt.figure(figsize=(6, 6))
    total.plot.pie(autopct='%1.1f%%', startangle=90, colors=sns.color_palette("Set2"))
    plt.title("Total Marks by Subject")
    plt.ylabel("")
    plt.tight_layout()
    plt.show()

# Main pipeline
df = load_and_clean_data(CSV_FILE)
blockchain = build_blockchain(df)
blockchain_df = blockchain_to_df(blockchain)

print("\nBlockchain Overview:")
display(blockchain_df)

df_with_trends = analyze_performance(df)
print("\nStudent Performance vs Average:")
display(df_with_trends.sort_values(by=["Student", "Exam"]))

plot_performance(df_with_trends)
plot_subject_pie(df_with_trends)

# Example: search_student_by_name("Alice", blockchain)
